## Example for NaCl electronic band structure using Conquest
> Basic example for how to use ASE to compute and plot a band structure of a non-magnetic insulator

In [ ]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt
import pickle

from distutils.spawn import find_executable
from numpy import amax, amin, size, array, trapz, floor, shape

from scipy.integrate import cumulative_trapezoid, trapezoid

from ase.units import Hartree
from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.io.conquest import Conquest_orthorhombic_check
from ase.io.conquest import get_fermi_level
#from ase.io.conquest import read_conquest

from ase.dft.bandgap import bandgap
from ase.dft.dos import DOS
from ase.visualize import view
from ase.spacegroup import get_spacegroup
from ase import Atoms

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import get_gapwind

In [ ]:
%%bash 
ase --version

#### Check if visualisation tools are installed

In [ ]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_rocksalt_bands'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

#### Define Conquest environment

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-develop/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/CONQUEST-develop/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-develop/tools/BasisGeneration/MakeIonFiles'

#### Setup Conquest basis set

In [ ]:
basis = {'Na' : {'gen_basis'            : True,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann'},
         'Cl' : {'gen_basis'            : False,
                 'basis_size'           : 'small',
                 'pseudopotential_type' : 'hamann'}
        }

val_elec = {'Na': 9, 'Cl': 7}

#### Generate rocksalt cell from CIF file data

- Choose either primitive or conventional cell

In [ ]:
primitive = True

- Generate rocksalt struture with $a=5.71$ Ang.

In [ ]:
if ( primitive ):
    # For primitive cell
    struct = bulk('NaCl', crystalstructure='rocksalt', a=5.71, cubic=False)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # warning must be printed and 'struct' is modified to conventional cell
    
else:
    # For conventional cell
    struct = bulk('NaCl', crystalstructure='rocksalt', a=5.71, cubic=True)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # no warning...

#### Compute the number of states
- Given `struct`and the `val_elec`compute the total number of electrons

In [ ]:
# Get atom labels
atom_names = struct.get_chemical_symbols()

# Compute total valence electrons
tot_val = 0
for i in range(len(struct)):
    for key, val in val_elec.items():
        if ( key == atom_names[i] ):
            tot_val = tot_val + val

print('Number of valence electron in the unit cell = ', tot_val)

- Compute number(s) of occupied states

In [ ]:
n_occ = []
if ( tot_val % 2 == 0):
    # paired electrons
    n_occ.append(int(tot_val/2))
else:
    # unpaired electrons
    n_occ.append(int((tot_val-1)/2))
    n_occ.append(int((tot_val+1)/2))
    
print()
print('Number of occupied bands = ', n_occ)

#### Extract and print main structural data from Atoms object (here `struct`)

In [ ]:
print_struct_data(struct,verbose=1)

- Space group ; Brillouin zone of space group [225](https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-kv-list?gnum=225&fig=fm3qmf&what=data)

In [ ]:
sg = get_spacegroup(struct)
print('Space group: {} ({}) '.format(sg.symbol,sg.no))

- Save input structure as VASP `POSCAR` file & `CIF` file (for checking purpose)

In [ ]:
struct.write(working_directory+'/input.vasp')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input (if possible) from `input.cif`

In [ ]:
#if ( 'VESTA' in cmd_app ):
#    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

#### Setup calculation using Conquest as calculator

In [ ]:
cutoff  =  80.0
kpoints = [3,3,3]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile' : 'True',
                  'IO.DumpChargeDensity' : 'True'}

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

#### Launch electronic structure calculation (be patient...)

In [ ]:
cq_dft_energy = struct.get_potential_energy()

#### Backup SCF files

- copy first calculation input/output files 

In [ ]:
subprocess.run(['cp', working_directory+'/'+'Conquest_input', 
                working_directory+'/'+'Conquest_input_scf'])

subprocess.run(['cp', working_directory+'/'+'Conquest_out', 
                working_directory+'/'+'Conquest_out_scf'])

subprocess.run(['cp', working_directory+'/'+'Conquest_out_ase', 
                working_directory+'/'+'Conquest_out_ase_scf'])

#### Read results (from `Conquest_out_ase` file)

In [ ]:
struct.calc.read_results()

#### Extract Fermi energy from SCF 

In [ ]:
fermi_energy_scf = struct.calc.eFermi

#### Band-path print

In [ ]:
latt  = struct.cell.get_bravais_lattice()
print(latt.description())
#print('lattice = ', latt)
#print('special k-point = ',list(latt.get_special_points()))

primitive = False

if ( primitive ):
    # For primitive cell
    path = 'WLGXWK'
else:
    # For conventional cell
    path = 'GXMGRX,MR'

#latt.plot_bz(show=True,path=path)

#print('k-point array shape =',size1,'x',size2)
#print('k-point coord list (fractional coord.)')
#print(kpath.kpts) 

#### Band structure calculation setup

In [ ]:
conquest_flags = {'General.LoadRho' : True, 'IO.DumpChargeDensity' : False}
nkpoints = 100 

- Setup kpoint mesh in Conquest bands input

In [ ]:
path  = 'GXRM'
kpath = struct.cell.bandpath(path, npoints=nkpoints)
print(kpath)
Changed_parameters = calc.set(kpts=kpath.kpts,self_consistent=False,**conquest_flags)

- ...or Setup kpoint lines in Conquest bands input

In [ ]:
#kpts_lines = {'path': 'GXRM', 'npoints': nkpoints, 'special_points' : '{GXRM}'}
#changed_parameters = calc.set(kpts=kpts_lines,self_consistent=False,**conquest_flags)

#### Run band structure calculation (be patient...)

In [ ]:
calc.calculate(struct)

#### Extract band structure and setup energy rescaling

In [ ]:
bs = calc.band_structure()

#### Extract higher occupied (ho) and lower unoccupied (lu) energy level from bands
> works only non-magnetic insulators

In [ ]:
ho, lu = get_gapwind( calc, kpath, n_occ )
print('higher   occupied energy = {:.4f} eV @kpt {} => {}'.format(ho[0],ho[1],kpath.kpts[ho[1]]))
print('lowest unoccupied energy = {:.4f} eV @kpt {} => {}'.format(lu[0],lu[1],kpath.kpts[lu[1]]))
print('band gap = {:.4f} eV'.format(lu[0] - ho[0]))
fermi_energy_bands = ho[0]

#### Choose wich Fermi level to use for bands and DOS post-processing 

> you have choice between `fermi_energy_scf` and `fermi_energy_bands`. For insulator, the first one is evaluated to be within the gap. The second one is setup at the highest occupied energy level.

In [ ]:
print('fermi SCF   = {:12.4f} eV'.format(fermi_energy_scf))
print('fermi bands = {:12.4f} eV'.format(fermi_energy_bands))
fermi_energy = fermi_energy_scf
fermi_energy = fermi_energy_bands

#### Print Fermi energy and energy spectrum bounds

In [ ]:
print('fermi = {:12.4f} eV'.format(fermi_energy))
e_max = amax(bs.energies) ; print('e_max = {:12.4f} eV'.format(e_max))
e_min = amin(bs.energies) ; print('e_min = {:12.4f} eV'.format(e_min))

#### Setup energy reference for band structure plot

In [ ]:
bs._reference = fermi_energy

# Rescaled energies if needed : Fermi energy setup to 0.0 !
bs._energies  = bs._energies  - fermi_energy
bs._reference = bs._reference - fermi_energy

#### Compute DOS ; Fermi energy must be given

In [ ]:
dos = DOS(calc, width=0.1, fermi=[fermi_energy-bs._reference],npts=1201)
d = dos.get_dos() / 2 # correct for ASE spin unpolarized
e = dos.get_energies() 

#### Print band structure and DOS

In [ ]:
%matplotlib notebook

# integrate DOS
sum_dos = trapezoid(d,e)

# Choose energy window [e_min, e_max]
e_max =  15.0
e_min = -25.0

# Set ax0 and ax1 on 1 single figure
fig, (ax0, ax1) = plt.subplots(1,2)

# Plot DOS on ax1
ax1.plot(d, e, color='black')

# Compute integrated DOS
sum_dos_cum = cumulative_trapezoid(d,e)

# Plot integrated DOS
ax1.plot(sum_dos_cum, e[:-1], color='black')

# Set plots x- and y- limits
ax1.set_ylim(e_min,e_max)
ax1.set_xlim(0.0,n_occ[0])

# Plot horizontal and vertical lines and x-axis label
ax1.axhline(bs.reference, color='k', ls=':')
ax1.axvline(n_occ[0], color='k', ls=':', linewidth=0.2)
ax1.set_xlabel('state count')

# Plot band structure on ax0
ax0.set_xlabel('$k$-path')
bs.plot(ax0,emin=e_min, emax=e_max, show=True,color='black')

#### Save figure as `bands.png` in ` working_directory`

In [ ]:
fig.savefig(working_directory+'/'+'bands.png', dpi=300, format='png', transparent=True)

#### Integrate DOS (checking purpose)

In [ ]:
# integrate DOS using trapezoidal rule
sum_dos = trapz(d,e)

e_ = e[0]
# integrate DOS up to Fermi level
while( e_ <= bs._reference ):
        e_ = e[i+1]
        i += 1
sum_dos_ef = trapezoid(d[0:i],e[0:i]) 

# print data
print('integrated DOS up to {:12.4f} = {:.4f}'.format(e[-1],sum_dos))
print('integrated DOS up to {:12.4f} = {:.4f}'.format(bs._reference,sum_dos_ef))

#### Compute direct and indirect band gap

In [ ]:
#WARNING: accuracy issue from ASE 'bandgap' (not yet resolved)
#solution: add epsilon to bs._reference (fermi_energy)
eps = 1e-6
gap, p1, p2 = bandgap(calc,efermi=fermi_energy+eps,direct=True)
gap, p1, p2 = bandgap(calc,efermi=fermi_energy+eps,direct=False)